<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/final_analysis_undefined_presence_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy matplotlib seaborn umap-learn scikit-learn

In [ ]:
#!/usr/bin/env python3
"""
final_analysis_undefined_presence_ai.py

1. Synthetic dataset (6 → 3)
2. MC‐Dropout model
3. Physics‐ and ODE‐informed residual losses
4. AdamW training with scheduler, clipping, early stop
5. Track per‐component losses
6. MC‐Dropout inference (mean/std)
7. Loss curves + per‐component loss curves
8. OOD detection AUROC + uncertainty plots
9. Reliability diagram
10. UMAP embedding colored by true presence
11. Physics residual histogram
"""

import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.autograd import grad
import umap
from sklearn.metrics import roc_auc_score

# 1. Reproducibility & Device
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Dataset
class UndefinedPresenceDataset(Dataset):
    def __init__(self, n=5000):
        u = np.random.uniform(-1,1,(n,1))
        v = np.random.uniform(0,2,(n,1))
        w = np.random.uniform(-2,2,(n,1))
        x = np.random.uniform(0,5,(n,1))
        y = np.random.uniform(-1,1,(n,1))
        z = np.random.uniform(0,1,(n,1))
        X = np.hstack([u,v,w,x,y,z]).astype(np.float32)

        presence = np.sin(u)*v + np.cos(w)
        dissolution = np.exp(-x*y)
        transcendence = z*(presence + dissolution)
        Y = np.hstack([presence,dissolution,transcendence]).astype(np.float32)
        Y += 0.01 * Y.std(axis=0) * np.random.randn(*Y.shape).astype(np.float32)

        self.X_mean, self.X_std = X.mean(0), X.std(0) + 1e-8
        self.Y_mean, self.Y_std = Y.mean(0), Y.std(0) + 1e-8

        self.X = ((X - self.X_mean)/self.X_std).astype(np.float32)
        self.Y = ((Y - self.Y_mean)/self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return torch.from_numpy(self.X[i]), torch.from_numpy(self.Y[i])

# 3. MC‐Dropout Model
class UndefinedPresenceAI(nn.Module):
    def __init__(self, inp=6, hid=32, out=3, p_drop=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(inp,hid),
            nn.ReLU(),
            nn.Dropout(p_drop),
            nn.Linear(hid,out)
        )

    def forward(self, x):
        return self.net(x)

# 4a. Physics residual
def physics_residual(pred, X, stats):
    Xd = X*stats['X_std'] + stats['X_mean']
    u,v,w,x,y,z = Xd.T
    pres = torch.sin(u)*v + torch.cos(w)
    dis  = torch.exp(-x*y)
    tr   = z*(pres+dis)
    Y_phys = torch.stack([pres,dis,tr],dim=1)
    Yn = (Y_phys - stats['Y_mean'])/stats['Y_std']
    return nn.MSELoss()(pred, Yn)

# 4b. ODE residual: ∂presence/∂u ≈ v⋅cos(u)
def ode_residual(pred, X, stats):
    # denormalize pred
    pred_den = pred*stats['Y_std'] + stats['Y_mean']
    pres = pred_den[:,0]
    grads = grad(pres.sum(), X, create_graph=True)[0]
    dp_du = grads[:,0]
    Xd = X*stats['X_std'] + stats['X_mean']
    u,v = Xd[:,0], Xd[:,1]
    target = v*torch.cos(u)
    return nn.MSELoss()(dp_du, target)

# 4c. Combined loss
def total_loss(pred, y_true, X, stats, lam_phys=1.0, lam_ode=0.5):
    mse  = nn.MSELoss()(pred, y_true)
    phys = physics_residual(pred, X, stats)
    ode  = ode_residual(pred, X, stats)
    return mse + lam_phys*phys + lam_ode*ode, mse.item(), phys.item(), ode.item()

# 5. MC‐Dropout prediction
def mc_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    S = torch.stack(preds)
    return S.mean(0), S.std(0)

# 6. Training with per‐component loss tracking
def train(model, dl_tr, dl_va, stats,
          epochs=100, lr=1e-3, wd=1e-5, patience=10):
    model.to(DEVICE)
    opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sch = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', factor=0.5, patience=5)
    best_val = float('inf'); wait = 0

    history = {k:[] for k in [
        'train_total','train_mse','train_phys','train_ode',
        'val_total','val_mse','val_phys','val_ode'
    ]}

    for epoch in range(1, epochs+1):
        # — Training —
        model.train()
        accum = np.zeros(4); count = 0
        for Xb, Yb in dl_tr:
            Xb = Xb.to(DEVICE).requires_grad_(True)
            Yb = Yb.to(DEVICE)
            pred = model(Xb)
            loss, m, p, o = total_loss(pred, Yb, Xb, stats)
            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(),1.0)
            opt.step()
            b = Xb.size(0)
            accum += np.array([loss.item(),m,p,o])*b
            count += b

        # store train losses
        history['train_total'].append(accum[0]/count)
        history['train_mse'].append(  accum[1]/count)
        history['train_phys'].append(accum[2]/count)
        history['train_ode'].append(  accum[3]/count)

        # — Validation (no torch.no_grad, we need grad for ODE) —
        model.eval()
        accum = np.zeros(4); count = 0
        for Xv, Yv in dl_va:
            Xv = Xv.to(DEVICE).requires_grad_(True)
            Yv = Yv.to(DEVICE)
            pred = model(Xv)
            loss, m, p, o = total_loss(pred, Yv, Xv, stats)
            b = Xv.size(0)
            accum += np.array([loss.item(),m,p,o])*b
            count += b

        history['val_total'].append(accum[0]/count)
        history['val_mse'].append(  accum[1]/count)
        history['val_phys'].append(accum[2]/count)
        history['val_ode'].append(  accum[3]/count)

        sch.step(history['val_total'][-1])
        print(f"Epoch {epoch:3d} | "
              f"Train {history['train_total'][-1]:.4f} | "
              f"Val   {history['val_total'][-1]:.4f}")

        if history['val_total'][-1] < best_val - 1e-6:
            best_val, wait = history['val_total'][-1], 0
            torch.save(model.state_dict(), "best_model.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    model.load_state_dict(torch.load("best_model.pth"))
    return history, model

# 7. Main & Analysis
if __name__ == "__main__":
    # Prepare data
    ds = UndefinedPresenceDataset()
    n_val = int(0.2*len(ds))
    ds_tr, ds_va = random_split(ds, [len(ds)-n_val, n_val])
    dl_tr = DataLoader(ds_tr, batch_size=128, shuffle=True)
    dl_va = DataLoader(ds_va, batch_size=256)

    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=DEVICE),
        'X_std':  torch.tensor(ds.X_std,  device=DEVICE),
        'Y_mean': torch.tensor(ds.Y_mean, device=DEVICE),
        'Y_std':  torch.tensor(ds.Y_std,  device=DEVICE),
    }

    # Train
    model = UndefinedPresenceAI().to(DEVICE)
    history, model = train(model, dl_tr, dl_va, stats)
    np.savez("history.npz", **history)

    # MC‐Dropout on in‐distribution
    X_all = torch.from_numpy(ds.X).to(DEVICE)
    mean_pred, std_pred = mc_predict(model, X_all)

    # Setup plots folder
    os.makedirs("plots", exist_ok=True)
    epochs = range(1, len(history['train_total'])+1)

    # 8. Plot: total loss curves
    plt.figure()
    plt.plot(epochs, history['train_total'], '--', label="Train Total")
    plt.plot(epochs, history['val_total'],   '-',  label="Val   Total")
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.title("Training & Validation Total Loss")
    plt.legend()
    plt.savefig("plots/loss_curves.png", dpi=150)

    # 9. Per‐component loss curves
    plt.figure()
    for comp in ['mse','phys','ode']:
        plt.plot(epochs, history[f"train_{comp}"], '--', label=f"Train {comp.upper()}")
        plt.plot(epochs, history[f"val_{comp}"],   '-',  label=f"Val   {comp.upper()}")
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.title("Per‐Component Loss Curves")
    plt.legend()
    plt.savefig("plots/per_component_losses.png", dpi=150)

    # 10. OOD detection AUROC
    def sample_ood(n=2000):
        u = np.random.uniform(-2,2,(n,1))
        v = np.random.uniform(0,2,(n,1))
        w = np.random.uniform(-2,2,(n,1))
        x = np.random.uniform(0,5,(n,1))
        y = np.random.uniform(-1,1,(n,1))
        z = np.random.uniform(0,1,(n,1))
        X = np.hstack([u,v,w,x,y,z]).astype(np.float32)
        X_std = (X - ds.X_mean)/ds.X_std
        return torch.from_numpy(X_std).to(DEVICE)

    X_ood = sample_ood()
    _, std_ood = mc_predict(model, X_ood)

    labels = np.concatenate([np.zeros(std_pred.size(0)), np.ones(std_ood.size(0))])
    print("\nOOD Detection AUROC:")
    for i,name in enumerate(['Presence','Dissolution','Transcendence']):
        scores = np.concatenate([std_pred[:,i].cpu().numpy(),
                                 std_ood[:,i].cpu().numpy()])
        auc = roc_auc_score(labels, scores)
        print(f"  {name:13s}: {auc:.4f}")

    # Plot ID vs OOD uncertainty
    plt.figure()
    for i,name in enumerate(['Presence','Dissolution','Transcendence']):
        sns.kdeplot(std_pred[:,i].cpu(), label=f"ID {name}")
        sns.kdeplot(std_ood[:,i].cpu(), label=f"OOD {name}")
    plt.title("ID vs OOD Uncertainty Distributions")
    plt.legend()
    plt.savefig("plots/ood_detection.png", dpi=150)

    # 11. Reliability diagram
    errors = (mean_pred - torch.from_numpy(ds.Y).to(DEVICE)).abs().cpu().numpy()
    stds   = std_pred.cpu().numpy()
    plt.figure()
    for i,name in enumerate(['Presence','Dissolution','Transcendence']):
        bins = np.linspace(stds[:,i].min(), stds[:,i].max(), 10)
        idx = np.digitize(stds[:,i], bins)-1
        avg_err, avg_std = [], []
        for b in range(len(bins)):
            mask = idx==b
            if mask.sum()>0:
                avg_err.append(errors[mask,i].mean())
                avg_std.append(stds[mask,i].mean())
        plt.plot(avg_std, avg_err, '-o', label=name)
    plt.xlabel("Avg Pred STD"); plt.ylabel("Avg Abs Error")
    plt.title("Reliability Diagram")
    plt.legend()
    plt.savefig("plots/reliability.png", dpi=150)

    # 12. UMAP embedding colored by true presence
    model.eval()
    with torch.no_grad():
        feats = model.net[0](X_all).cpu().numpy()
    emb = umap.UMAP(n_components=2, random_state=SEED).fit_transform(feats)
    raw = ds.X*ds.X_std + ds.X_mean
    u,v,w = raw[:,0], raw[:,1], raw[:,2]
    true_pres = np.sin(u)*v + np.cos(w)
    plt.figure(figsize=(6,5))
    sc = plt.scatter(emb[:,0], emb[:,1], c=true_pres, cmap="coolwarm", s=4)
    plt.colorbar(sc, label="True Presence")
    plt.title("UMAP of Hidden Features")
    plt.savefig("plots/umap_presence.png", dpi=150)

    # 13. Physics residual histogram
    with torch.no_grad():
        Xd = X_all*stats['X_std'] + stats['X_mean']
        u,v,w,x,y,z = Xd.T
        pres = torch.sin(u)*v + torch.cos(w)
        dis  = torch.exp(-x*y)
        tr   = z*(pres+dis)
        Y_phys = torch.stack([pres,dis,tr],dim=1).cpu().numpy()
    pred_den = mean_pred.cpu().numpy()*ds.Y_std + ds.Y_mean
    residuals = ((pred_den - Y_phys)**2).mean(axis=1)
    plt.figure()
    sns.histplot(residuals, bins=50, kde=True)
    plt.title("Physics Residual (MSE) Histogram")
    plt.xlabel("Residual"); plt.ylabel("Count")
    plt.savefig("plots/physics_residual_hist.png", dpi=150)

    print("\nDone. All plots in ./plots/ and history saved to history.npz")